In [1]:
#import stuff
import pandas as pd
from sklearn.ensemble import  GradientBoostingRegressor
import numpy as np
from sklearn import ensemble , preprocessing
import  xgboost as xgb
import time
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn import  svm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
import matplotlib.pyplot  as plt
from IPython.display import display
from sklearn.grid_search import GridSearchCV
from __future__ import division
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
#define root mean  squared loss   error metric  used in the competition
def  rmsle(y,yhat):
    from math import log , sqrt
    assert len(y)==len(yhat)
    yhat[yhat<0]=0
    rmsle=0
    for pair in zip(y,yhat):
        logdiff=log(1+pair[1])-log(1+pair[0])
        rmsle+=logdiff*logdiff
      
    rmsle=sqrt(rmsle/len(yhat))
    return rmsle
        
from sklearn.metrics import make_scorer    
rmsle_loss=make_scorer(rmsle,greater_is_beter=False)

In [11]:
#read  data
train=pd.read_csv('d:/buldozer/generateTrain.csv')
test=pd.read_csv('d:/buldozer/generateTest.csv')
cost=train['cost']
test=test.drop(['tube_assembly_id','component_id_7','component_id_8','quantity_8','quantity_7','quantity_6','quantity_5'],axis=1)
train=train.drop(['cost','tube_assembly_id','component_id_7','component_id_8','quantity_8','quantity_7','quantity_6','quantity_5'],axis=1)
trainvec=np.array(train)
testvec=np.array(test)
cost=np.array(cost)
print(train.shape)
print(test.shape)
               


(30213, 37)
(30235, 37)


In [4]:
#read second data set in order to  tune it   on cross validation and prepare it  for   generating   same learners
#but on  different  data set vor  diversity
trainvec2=pd.read_csv('d:/buldozer1/trainvec2.csv')
testvec2=pd.read_csv('d:/buldozer1/testvec2.csv')
cost2=pd.read_csv('d:/buldozer1/cost2.csv')
trainvec2=np.array(trainvec2)
testvec2=np.array(testvec2)
cost2=np.array(cost2)
print(trainvec2.shape)
print(testvec2.shape)
print(cost2.shape)

(30213L, 38L)
(30235L, 38L)
(30213L, 1L)


In [5]:

def ada_learning(y, train, test,trees):
    label_log = np.log1p(y)
    #1'd model
    #DecisionTreeRegressor= DecisionTreeRegressor(max_depth=4)
    ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15),
                          n_estimators=trees)   
    ada.fit(train, label_log)
    p=np.expm1(ada.predict(test))
    return p

In [107]:
#define  xgboost function
def x_b(y, train, test,trees):
    labels=np.log1p(y)
    params={}
    params["objective"] = "reg:linear"
    params["eta"] = 0.02
    params["min_child_weight"] = 7
    params["subsample"] = 0.7
    params["colsample_bytree"] = 0.65
    params["scale_pos_weight"] = 0.5
    params["silent"] = 1
    params["max_depth"] = 7
    params["seed"]=2
    plst = list(params.items())
    xgtrain=xgb.DMatrix(train, label=labels)
    xgtest = xgb.DMatrix(test)
    num_rounds = trees
    model = xgb.train(plst, xgtrain, num_rounds)
    prediction=model.predict(xgtest)
    prediction=np.expm1(prediction)
    return prediction
    
    
    

In [123]:
#define random forest   regressors 
def  random_forest(y,train , test, trees):
    labels=np.log1p(y)
    rf=RandomForestRegressor(n_estimators=trees,n_jobs=-1)
    rf.fit(train,labels)
    p=rf.predict(test)
    p=np.expm1(p)
    return p

In [8]:
def  random_forest(y,train , test, trees):
    labels=np.log1p(y)
    rf=RandomForestRegressor(n_estimators=trees,n_jobs=-1)
    rf.fit(train,labels)
    p=rf.predict(test)
    p=np.expm1(p)
    return p

In [124]:
#define  gradient boosting  machines
def  gbm(y,train ,test,trees,max_depth):
    labels=np.log1p(y)
    params = {'n_estimators': trees, 'max_depth': max_depth, 'min_samples_split': 6,
          'learning_rate': 0.01, 'loss': 'lad', 'subsample': 0.7}
    gbr = GradientBoostingRegressor(**params)
    gbr.fit(train,labels)
    prediction=gbr.predict(test)
    predictions=np.expm1(prediction)
 
    return predictions


In [207]:
#look   how scores  are  going on on  cross validation   according to  number  of  trees
cross_error=[]
rand=np.random.RandomState(1234)
for cv in  range(1,5):
    idx_tr,idx_ts=train_test_split(np.arange(train1.shape[0]),train_size=0.3,random_state=rand)
    cv_train=train1[idx_tr]
    cv_test=train1[idx_ts]
    cv_train_y=cost[idx_tr]
    cv_test_y=cost[idx_ts]
    #pred_cv=random_forest(cv_train_y,cv_train,cv_test,1000)
    pred_cv1=x_b(cv_train_y,cv_train,cv_test,8000)
    pred_cv2=x_b(cv_train_y,cv_train,cv_test,5000)
    pred_cv3=x_b(cv_train_y,cv_train,cv_test,6000)
    pred_cv4=x_b(cv_train_y,cv_train,cv_test,7000)
    pred_cv5=x_b(cv_train_y,cv_train,cv_test,4000)
    pred_cv6=x_b(cv_train_y,cv_train,cv_test,9000)
    pred_cv=(pred_cv1+pred_cv2+pred_cv3+pred_cv4+pred_cv5+pred_cv6)/6
    #pred_cv2=gbm(cv_train_y,cv_train,cv_test,5000,13)
  
  
    err=rmsle(pred_cv,cv_test_y)
    cross_error.append(err)
print("eroare ",cross_error)    
print("mean of error",np.mean(cross_error))    
                                      

('eroare ', [0.28767955785945454, 0.2951165063996632, 0.2940179846214483, 0.2866478754034704])
('mean of error', 0.29086548107100912)


In [1]:
#getting weights  for  small ensamble of  2 learners
#split data
rand=np.random.RandomState(1234)
train_index,test_index=train_test_split(np.arange(trainvec.shape[0]),train_size=0.3,random_state=rand)
train_x,train_y=trainvec[train_index],cost[train_index]
test_x,test_y=trainvec[test_index],cost[test_index]
#build  the regressor learners
predictions=[]
 
#for  data set  2
predictions.append(x_b(train_y[:,0],train_x,test_x,1000))
predictions.append(x_b(train_y[:,0],train_x,test_x,2000))
predictions.append(x_b(train_y[:,0],train_x,test_x,3000))
predictions.append(x_b(train_y[:,0],train_x,test_x,4000))
predictions.append(x_b(train_y[:,0],train_x,test_x,5500))
predictions.append(ada_learning(train_y[:,0],train_x,test_x,100))
predictions.append(ada_learning(train_y[:,0],train_x,test_x,50))
predictions.append(ada_learning(train_y[:,0],train_x,test_x,300))
predictions.append(ada_learning(train_y[:,0],train_x,test_x,500))
predictions.append(random_forest(train_y[:,0],train_x,test_x,50))
predictions.append(random_forest(train_y[:,0],train_x,test_x,100))
predictions.append(random_forest(train_y[:,0],train_x,test_x,500))
predictions.append(gbm(train_y[:,0],train_x,test_x,4000,10))
predictions.append(gbm(train_y[:,0],train_x,test_x,2000,12))
predictions.append(gbm(train_y[:,0],train_x,test_x,5000,9))
predictions.append(gbm(train_y[:,0],train_x,test_x,1000,9))    
predictions.append(gbm(train_y[:,0],train_x,test_x,750,7))

 


NameError: name 'np' is not defined

In [93]:
#define  the  wheighted  root mean squared logharitmic  metric
pred=len(predictions)
np.set_printoptions(suppress=True)
from scipy.optimize import  minimize
from scipy import optimize
from scipy.optimize import  leastsq
def weighted_rmsle(weights):
    final_prediction=0
    for weight,prediction in zip(weights,predictions):
        final_prediction+=weight*prediction
    
 
    return rmsle(final_prediction,cost)
    # return mean_squared_error(final_prediction,test_y)
#set starting  values
starting_values=[0.1]*len(predictions)
#set constraints
cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
#set weights bound  between  0 and  1 open interval
bounds=[(0,1)]*len(predictions)
res=minimize(weighted_rmsle,starting_values,method='SLSQP',bounds=bounds )
 

 
print('Ensamble Score: {best_score}'.format(best_score=res['fun']))
print('Best Weights: {weights}'.format(weights=res['x']))
weights=res['x']
weights 
 

Ensamble Score: 1.07492425876
Best Weights: [ 0.          0.          0.         -0.          0.         -0.         -0.
  0.          0.88720023 -0.         -0.         -0.          0.         -0.
 -0.         -0.          0.          0.          0.         -0.          0.
  0.         -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.          0.        ]


array([ 0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.88720023, -0.        ,
       -0.        , -0.        ,  0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        ,  0.        ])

In [138]:
predictions=[]
newTrain=pd.read_csv('d:/buldozer/newTrain.csv')
newTest=pd.read_csv('d:/buldozer/newTest.csv')











In [192]:
train1=np.hstack((newTrain,train))
test1=np.hstack((newTest,test))

In [208]:
p1=x_b(cost,train1,test1,4000)
p2=x_b(cost,train1,test1,5000)
p3=x_b(cost,train1,test1,6000)
p4=x_b(cost,train1,test1,7000)
p5=x_b(cost,train1,test1,8000)
p6=x_b(cost,train1,test1,9000)
p=(p1+p2+p3+p4+p5+p6)/6


In [211]:
pd.DataFrame(p).to_csv('d:/buldozer/things3.csv')

In [209]:
pd.DataFrame(p).describe()

,0
count,30235.000000
mean,12.682378
std,24.397954
min,0.721025
25%,3.978690
50%,6.613832
75%,13.242001
max,676.477905
